In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from electricitylci import get_generation_process_df, get_generation_mix_process_df
from electricitylci.consumption_mix import check_trading_normalized, trading_mix_fuels
from electricitylci.globals import data_dir

from os.path import join

Select a model number to use:
	1: ELCI_2
	2: ELCI_1
	3: ELCI_3
	4: ELCI_5


 2


Model ELCI_1 selected.


## Test data

In [3]:
regions = [x for x in 'ABCD']

trade = {
    'A': [1, 0, 0, 0],
    'B': [0, .5, .25, .25],
    'C': [0, .1, .9, 0],
    'D': [0, 0, .5, .5]
}
trade_df = pd.DataFrame(data=trade, index=regions)

fuels = ['gas', 'wind']

gen_df = pd.DataFrame(
    index = pd.MultiIndex.from_product([regions, fuels],
                                       names=['Subregion', 'FuelCategory']),
    columns = ['Generation_Ratio'],
    data = [1, 0, 0, 1, .5, .5, .1, .9]
).reset_index()

In [4]:
gen_df

,Subregion,FuelCategory,Generation_Ratio
0,A,gas,1.0
1,A,wind,0.0
2,B,gas,0.0
3,B,wind,1.0
4,C,gas,0.5
5,C,wind,0.5
6,D,gas,0.1
7,D,wind,0.9


In [5]:
trade_df

,A,B,C,D
A,1,0.00,0.0,0.0
B,0,0.50,0.1,0.0
C,0,0.25,0.9,0.5
D,0,0.25,0.0,0.5


In [6]:
print(trading_mix_fuels(gen_mix=gen_df, trading_matrix=trade_df))

   Subregion from_region FuelCategory  trading_gen_ratio
0          A           A          gas              1.000
1          B           B         wind              0.500
2          B           C          gas              0.125
3          B           C         wind              0.125
4          B           D          gas              0.025
5          B           D         wind              0.225
6          C           B         wind              0.100
7          C           C          gas              0.450
8          C           C         wind              0.450
9          D           C          gas              0.250
10         D           C         wind              0.250
11         D           D          gas              0.050
12         D           D         wind              0.450


In [7]:
trading_mix_fuels(gen_mix=gen_df, trading_matrix=trade_df)

,Subregion,from_region,FuelCategory,trading_gen_ratio
0,A,A,gas,1.000
1,B,B,wind,0.500
2,B,C,gas,0.125
3,B,C,wind,0.125
4,B,D,gas,0.025
5,B,D,wind,0.225
6,C,B,wind,0.100
7,C,C,gas,0.450
8,C,C,wind,0.450
9,D,C,gas,0.250


## BA name/code match

In [11]:
path = join(data_dir, 'BA code match.csv')
ba_match = pd.read_csv(path)

In [12]:
ba_match.set_index('ba_name', inplace=True)

## Trading data

In [13]:
trading_actual = pd.read_csv('trading_matrix_actual_region.csv', index_col=0)

In [14]:
trading_actual = pd.read_csv('trading_matrix_actual_BA.csv', index_col=1)
trading_actual = trading_actual.loc[:, 'AEC':]

In [15]:
trading_actual.tail()

,AEC,AECI,AVA,AZPS,BANC,BCHA,BPAT,CHPD,CISO,CPLE,...,TEPC,TIDC,TPWR,TVA,WACM,WALC,WAUE,WAUW,WWA,YAD
Source BAA,,,,,,,,,,,,,,,,,,,,,
WALC,0.0,0.000004,0.000000,205.748501,0.000000,0.0,0.000000,0.0,2.299981e+06,2.566513e-11,...,1.389344e+06,10012.137492,0.000000,5.329872e-07,0.792476,8.756409e+06,0.0,0.000000,0.0,0.0
WAUE,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0.000000,0.0,0.0
WAUW,0.0,1919.154157,0.000000,8269.596330,0.000000,0.0,0.000000,0.0,5.070989e+03,1.175174e-02,...,2.258085e+03,22.074721,0.000000,2.440482e+02,161094.415578,9.123140e+03,0.0,690867.500000,0.0,0.0
WWA,0.0,44.341214,20432.689099,1104.702939,9837.652922,0.0,94279.632738,0.0,2.692337e+04,2.715189e-04,...,1.684637e+02,206.585958,2698.554571,5.638625e+00,3725.533214,4.188278e+02,0.0,15962.183553,-523.0,0.0
YAD,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000e+00,1.032469e+04,...,0.000000e+00,0.000000,0.000000,6.771262e+00,0.000000,0.000000e+00,0.0,0.000000,0.0,49449.5


In [29]:
pd.Series(np.diag(trading_actual), index=[trading_actual.index, trading_actual.columns])

Source BAA      
AEC         AEC     4.544007e+06
AECI        AECI    1.958502e+07
AVA         AVA     9.846925e+06
AZPS        AZPS    3.127218e+07
BANC        BANC    1.729542e+07
BCHA        BCHA    0.000000e+00
BPAT        BPAT    6.615449e+07
CHPD        CHPD    3.286178e+06
CISO        CISO    2.061889e+08
CPLE        CPLE    6.083340e+07
CPLW        CPLW    4.945908e+06
DEAA        DEAA    0.000000e+00
DOPD        DOPD    1.174210e+06
DUK         DUK     1.045663e+08
EEI         EEI     3.530287e+04
EPE         EPE     8.333488e+06
ERCO        ERCO    3.482001e+08
FMPP        FMPP    1.753042e+07
FPC         FPC     5.298050e+07
FPL         FPL     1.245266e+08
GCPD        GCPD    4.746350e+06
GRID        GRID    2.931000e+03
GRIF        GRIF   -2.000000e+00
GRIS        GRIS    0.000000e+00
GRMA        GRMA    3.502000e+03
GVL         GVL     2.059152e+06
GWA         GWA    -9.025000e+02
HGMA        HGMA   -5.000000e-01
HQT         HQT     0.000000e+00
HST         HST     5.1755

In [16]:
check_trading_normalized(trading_actual)

In [18]:
trading_actual.tail(5)

,AEC,AECI,AVA,AZPS,BANC,BCHA,BPAT,CHPD,CISO,CPLE,...,TEPC,TIDC,TPWR,TVA,WACM,WALC,WAUE,WAUW,WWA,YAD
Source BAA,,,,,,,,,,,,,,,,,,,,,
WALC,0.0,1.910153e-13,0.000000,0.000005,0.000000,NaN,0.000000,0.0,0.008698,4.106204e-19,...,0.057564,0.002506,0.000000,3.088518e-15,2.939391e-08,0.559271,NaN,0.000000,-0.0,0.0
WAUE,0.0,0.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000,0.0,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN,0.000000,-0.0,0.0
WAUW,0.0,8.746351e-05,0.000000,0.000220,0.000000,NaN,0.000000,0.0,0.000019,1.880180e-10,...,0.000094,0.000006,0.000000,1.414194e-06,5.975188e-03,0.000583,NaN,0.563311,-0.0,0.0
WWA,0.0,2.020806e-06,0.001388,0.000029,0.000379,NaN,0.001138,0.0,0.000102,4.344072e-12,...,0.000007,0.000052,0.000378,3.267432e-08,1.381846e-04,0.000027,NaN,0.013015,1.0,0.0
YAD,0.0,0.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000,0.0,0.000000,1.651864e-04,...,0.000000,0.000000,0.000000,3.923765e-08,0.000000e+00,0.000000,NaN,0.000000,-0.0,1.0


In [23]:
trading_actual.iloc[:, 0]

Source_Region
California                                     0.742308
Carolinas                                      0.000000
Central                                        0.000000
Electric Reliability Council of Texas, Inc.    0.000000
Florida                                        0.000000
Mid-Atlantic                                   0.000000
Midwest                                        0.000000
New England ISO                                0.000000
New York Independent System Operator           0.000000
Northwest                                      0.141222
Southeast                                      0.000000
Southwest                                      0.116471
Tennessee Valley Authority                     0.000000
Name: California, dtype: float64

In [7]:
trading_norm = trading_actual.copy()

In [8]:
for col in trading_norm.columns:
    trading_norm[col] /= trading_norm[col].sum()

In [9]:
trading_norm.head()

,California,Carolinas,Central,"Electric Reliability Council of Texas, Inc.",Florida,Mid-Atlantic,Midwest,New England ISO,New York Independent System Operator,Northwest,Southeast,Southwest,Tennessee Valley Authority
Source_Region,,,,,,,,,,,,,
California,0.742308,7.693603e-15,1.517710e-09,0.0,4.759087e-18,1.043652e-14,1.023991e-11,1.539375e-17,5.288244e-16,0.001599,2.068744e-14,0.001834,1.290240e-12
Carolinas,0.000000,9.566665e-01,0.000000e+00,0.0,1.588492e-06,9.258602e-03,2.591913e-04,1.368911e-05,4.702644e-04,0.000000,6.945982e-03,0.000000,1.701044e-05
Central,0.000000,9.475246e-06,9.965565e-01,0.0,5.861171e-09,1.285335e-05,1.152934e-02,1.895855e-08,6.512867e-07,0.000000,2.547812e-05,0.000000,1.589027e-03
"Electric Reliability Council of Texas, Inc.",0.000000,1.838383e-09,3.626562e-04,1.0,1.137182e-12,2.493801e-09,2.446822e-06,3.678331e-12,1.263624e-10,0.000000,4.943255e-09,0.000000,3.083023e-07
Florida,0.000000,0.000000e+00,0.000000e+00,0.0,9.997743e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00


In [18]:
trading_norm.sum(axis='rows').sum() == 13

True

## Gen mix data

In [65]:
gen_mix = get_generation_mix_process_df(regions='BA')
gen_mix['Subregion'] = gen_mix['Subregion'].map(ba_match['ba_code'])

Actual generation data is used when replacing eGRID
Loading 2017 EIA-923 data from csv file
Loading 2017 EIA-860 plant data from csv file
Loading 2017 EIA-923 data from csv file
Loading 2017 EIA-860 plant data from csv file


In [30]:
gen_mix.columns

Index(['Subregion', 'FuelCategory', 'Electricity', 'Generation_Ratio'], dtype='object')

In [9]:
gen_mix.Subregion.unique()

array(['California', 'Carolinas', 'Central',
       'Electric Reliability Council of Texas, Inc.', 'Florida',
       'Mid-Atlantic', 'Midwest', 'New England ISO',
       'New York Independent System Operator', 'Northwest', 'Southeast',
       'Southwest', 'Tennessee Valley Authority'], dtype=object)

In [5]:
gen_mix.sample(5)

,Subregion,FuelCategory,Electricity,Generation_Ratio
295,Southwestern Power Administration,OIL,855.54,0.000118
22,Bonneville Power Administration,GAS,8484460.00,0.072965
23,Bonneville Power Administration,GEOTHERMAL,84436.00,0.000726
324,Western Area Power Administration - Rocky Moun...,GAS,3416746.94,0.077913
325,Western Area Power Administration - Rocky Moun...,HYDRO,1873242.00,0.042716


In [51]:
gen_mix.sample(5)

,Subregion,FuelCategory,Electricity,Generation_Ratio
252,NaN,SOLAR,7.784930e+05,0.011712
51,NaN,GAS,5.405380e+05,0.901196
198,NaN,GEOTHERMAL,1.567000e+03,0.000153
5,AZPS,WIND,3.839280e+05,0.024148
158,NYIS,HYDRO,3.013397e+07,0.230043


In [19]:
len(gen_mix.Subregion.unique())

51

In [20]:
trading_actual.shape

(74, 75)

In [ ]:
gen_mix.dropna()

In [36]:
set(gen_mix.Subregion.dropna().unique()).issubset(set(trading_actual.index))

True

In [33]:
set(gen_mix.Subregion.unique()) - set(trading_actual.index)

{nan}

In [39]:
%%time
results = trading_mix_fuels(gen_mix, trading_actual)

CPU times: user 20min 32s, sys: 12.4 s, total: 20min 44s
Wall time: 21min 46s


In [11]:
gen_mix.loc[gen_mix.Subregion == 'California', :]

,Subregion,FuelCategory,Electricity,Generation_Ratio
0,California,BIOMASS,5.102532e+06,0.025792
1,California,GAS,8.598807e+07,0.434641
2,California,GEOTHERMAL,1.197144e+07,0.060512
3,California,HYDRO,3.530088e+07,0.178434
4,California,NUCLEAR,1.790108e+07,0.090484
5,California,OFSL,6.652600e+04,0.000336
6,California,OIL,3.140752e+04,0.000159
7,California,OTHF,5.376168e+05,0.002717
8,California,SOLAR,2.749795e+07,0.138993
9,California,WIND,1.343936e+07,0.067932


In [13]:
set(gen_mix.Subregion.unique()) == set(trading_norm.index)

True

In [19]:
list(trading_norm.loc[trading_norm['California'] > 0, 'California'].iteritems())

[('California', 0.7423075562540813),
 ('Northwest', 0.1412218562062946),
 ('Southwest', 0.11647058753962414)]

In [28]:
gen_mix.head()

,Subregion,FuelCategory,Electricity,Generation_Ratio
0,California,BIOMASS,5.102532e+06,0.025792
1,California,GAS,8.598807e+07,0.434641
2,California,GEOTHERMAL,1.197144e+07,0.060512
3,California,HYDRO,3.530088e+07,0.178434
4,California,NUCLEAR,1.790108e+07,0.090484


In [40]:
trading_actual.head()

,AEC,AECI,AVA,AZPS,BANC,BCHA,BPAT,CHPD,CISO,CPLE,...,TEPC,TIDC,TPWR,TVA,WACM,WALC,WAUE,WAUW,WWA,YAD
Source BAA,,,,,,,,,,,,,,,,,,,,,
AEC,1.0,0.000000e+00,0.000000,0.000000e+00,0.000000,NaN,0.000000,0.0,0.000000,4.170836e-06,...,0.000000,0.000000,0.000000,4.529113e-05,0.000000e+00,0.000000,NaN,0.0,-0.0,0.0
AECI,0.0,8.925673e-01,0.000000,0.000000e+00,0.000000,NaN,0.000000,0.0,0.000000,1.178243e-06,...,0.000000,0.000000,0.000000,8.862260e-03,0.000000e+00,0.000000,NaN,0.0,-0.0,0.0
AVA,0.0,9.397841e-13,0.669058,2.689387e-05,0.000021,NaN,0.000063,0.0,0.000019,2.020229e-18,...,0.000019,0.000007,0.000021,1.519533e-14,1.446163e-07,0.000148,NaN,0.0,-0.0,0.0
AZPS,0.0,2.903284e-08,0.000000,8.308349e-01,0.000000,NaN,0.000000,0.0,0.040693,6.241112e-14,...,0.134546,0.011723,0.000000,4.694307e-10,4.467646e-03,0.159698,NaN,0.0,-0.0,0.0
BANC,0.0,1.294045e-17,0.000000,3.703178e-10,0.666805,NaN,0.000000,0.0,0.015923,2.781774e-23,...,0.000004,0.043916,0.000000,2.092335e-19,1.991309e-12,0.000038,NaN,0.0,-0.0,0.0


In [41]:
gen_mix.head()

,Subregion,FuelCategory,Electricity,Generation_Ratio
0,YAD,HYDRO,6.218880e+05,1.000000
1,AZPS,BIOMASS,1.714060e+05,0.010781
2,AZPS,GAS,3.366437e+06,0.211736
3,AZPS,SOLAR,1.836543e+06,0.115512
4,AZPS,SUB,1.014088e+07,0.637824


In [42]:
gen_mix.set_index('Subregion', inplace=True)

In [43]:
gen_mix['trading_amount'] = trading_actual['AZPS']

In [66]:
gen_mix.sample(10)

,Subregion,FuelCategory,Electricity,Generation_Ratio
33,CISO,NUCLEAR,1.790108e+07,0.105415
203,PGE,GAS,6.267484e+06,0.519919
98,NaN,WIND,3.307429e+06,0.033497
204,PGE,HYDRO,2.424309e+06,0.201108
75,NaN,BIT,1.137210e+05,0.001150
118,NaN,GAS,3.747762e+06,0.254387
162,NYIS,WIND,4.135951e+06,0.031574
169,NaN,SOLAR,1.363300e+04,0.000736
46,DUK,HYDRO,1.674078e+06,0.014620
66,ERCO,NUCLEAR,3.858140e+07,0.110585


In [79]:
def trading_mix_fuels_test(gen_mix, trading_norm):
    # slow but it works
#     assert set(gen_mix.Subregion.unique()) == set(trading_norm.index)
    
#     gen_mix['from_region'] = None
#     gen_mix['region'] = gen_mix['Subregion']
    _gen_mix = gen_mix.dropna().set_index('Subregion')
    print(_gen_mix.head())
    
#     fuels = gen_mix.FuelCategory.unique()
    regions = trading_norm.index
    print(regions[:5])
    
#     m_index = pd.MultiIndex.from_product(iterables=[fuels, regions])
#     base_cols = ['Generation_Ratio', 'trading_gen_ratio']
#     base_df = pd.DataFrame(index=m_index, columns=base_cols)
#     base_df.reset_index(inplace=True)
    
    df_list = []
    for region in regions:
#         input_regions = trading_norm.loc[
#             trading_norm[region] > 0, region
#         ]
        region_df = _gen_mix.copy()
        region_df['trading_amount'] = trading_norm[region]
        region_df['region'] = region
        region_df['trading_gen_ratio'] = (
            region_df['trading_amount']
            * region_df['Generation_Ratio']
        )
        
        if region == 'AZPS':
            print(region_df.sample(10))
        
        df_list.append(region_df)
        
#         for in_region, source_value in input_regions.iteritems():
#             in_fuels = gen_mix.loc[gen_mix.Subregion == in_region, :]
#             in_fuels['from_region'] = in_region
# #             in_fuels['region'] = region
#             in_fuels['trading_gen_ratio'] = (
#                 in_fuels['Generation_Ratio']
#                 * source_value
#             )
            
#             df_list.append(in_fuels)
            
    full_gen_df = pd.concat(df_list)
    full_gen_df['from_region'] = full_gen_df.index
    full_gen_df.dropna(inplace=True)
    full_gen_df.reset_index(drop=True, inplace=True)
    
    keep_cols = [
        'region',
        'from_region',
        'FuelCategory',
        'trading_gen_ratio',
    ]
    full_gen_df = full_gen_df.loc[
        full_gen_df['trading_gen_ratio'] > 0,
        keep_cols
    ]
    
    return full_gen_df
        

In [80]:
%%time
results = trading_mix_fuels_test(gen_mix=gen_mix, trading_norm=trading_actual)

          FuelCategory   Electricity  Generation_Ratio
Subregion                                             
YAD              HYDRO  6.218880e+05          1.000000
AZPS           BIOMASS  1.714060e+05          0.010781
AZPS               GAS  3.366437e+06          0.211736
AZPS             SOLAR  1.836543e+06          0.115512
AZPS               SUB  1.014088e+07          0.637824
Index(['AEC', 'AECI', 'AVA', 'AZPS', 'BANC'], dtype='object', name='Source BAA')
          FuelCategory   Electricity  Generation_Ratio  trading_amount region  \
Subregion                                                                       
IPCO              WIND  1.772777e+06      1.266752e-01    1.746624e-04   AZPS   
ERCO              WIND  6.149238e+07      1.762535e-01    0.000000e+00   AZPS   
LDWP             HYDRO  8.982100e+05      7.352442e-02    7.955023e-10   AZPS   
TVA                 RC  8.792838e+06      6.090412e-02    0.000000e+00   AZPS   
IPCO               OIL  1.800000e+01      1.2862

In [82]:
results.shape

(6520, 4)

In [83]:
results.loc[results['trading_gen_ratio'] > 0]

,region,from_region,FuelCategory,trading_gen_ratio
138,AEC,AEC,BIOMASS,5.218387e-03
139,AEC,AEC,BIT,2.386932e-01
140,AEC,AEC,GAS,7.532380e-01
141,AEC,AEC,HYDRO,2.850335e-03
232,AECI,AZPS,BIOMASS,3.129971e-10
233,AECI,AZPS,GAS,6.147306e-09
234,AECI,AZPS,SOLAR,3.353632e-09
235,AECI,AZPS,SUB,1.851783e-08
236,AECI,AZPS,WIND,7.010744e-10
237,AECI,DEAA,GAS,1.564605e-13


In [29]:
%load_ext line_profiler

In [31]:
%lprun -f trading_mix_fuels trading_mix_fuels(gen_mix, trading_norm)

Timer unit: 1e-06 s

Total time: 17.2271 s
File: <ipython-input-26-e4125fd1c779>
Function: trading_mix_fuels at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def trading_mix_fuels(gen_mix, trading_norm):
     2                                               # slow but it works
     3         1       1955.0   1955.0      0.0      assert set(gen_mix.Subregion.unique()) == set(trading_norm.index)
     4                                               
     5         1          2.0      2.0      0.0      df_list = []
     6        14        296.0     21.1      0.0      for region in gen_mix.Subregion.unique():
     7        13         40.0      3.1      0.0          input_regions = trading_norm.loc[
     8        13      12139.0    933.8      0.1              trading_norm[region] > 0, region
     9                                                   ]
    10                                                   
    11       

In [42]:
%lprun -f trading_mix_fuels trading_mix_fuels(gen_mix, trading_norm)

Timer unit: 1e-06 s

Total time: 12.3054 s
File: <ipython-input-41-23d09c551f79>
Function: trading_mix_fuels at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def trading_mix_fuels(gen_mix, trading_norm):
     2                                               # slow but it works
     3         1        676.0    676.0      0.0      assert set(gen_mix.Subregion.unique()) == set(trading_norm.index)
     4                                               
     5         1        543.0    543.0      0.0      gen_mix['from_region'] = None
     6         1        514.0    514.0      0.0      gen_mix['region'] = gen_mix['Subregion']
     7         1        746.0    746.0      0.0      gen_mix['trading_gen_ratio'] = 0
     8                                               
     9         1          3.0      3.0      0.0      df_list = []
    10        14       1086.0     77.6      0.0      for region in gen_mix.Subregion.unique()

In [35]:
%lprun -f trading_mix_fuels trading_mix_fuels(gen_mix, trading_norm)

Timer unit: 1e-06 s

Total time: 11.7265 s
File: <ipython-input-34-23d09c551f79>
Function: trading_mix_fuels at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def trading_mix_fuels(gen_mix, trading_norm):
     2                                               # slow but it works
     3         1        618.0    618.0      0.0      assert set(gen_mix.Subregion.unique()) == set(trading_norm.index)
     4                                               
     5         1        504.0    504.0      0.0      gen_mix['from_region'] = None
     6         1        526.0    526.0      0.0      gen_mix['region'] = gen_mix['Subregion']
     7         1        350.0    350.0      0.0      gen_mix['trading_gen_ratio'] = 0
     8                                               
     9         1          2.0      2.0      0.0      df_list = []
    10        14        578.0     41.3      0.0      for region in gen_mix.Subregion.unique()

In [21]:
df = trading_mix_fuels(gen_mix, trading_norm)

In [23]:
df.sample(10)

,region,from_region,FuelCategory,trading_gen_ratio
33,Mid-Atlantic,"Electric Reliability Council of Texas, Inc.",LIG,1.924561e-10
23,Central,Central,NUCLEAR,6.994909e-02
67,Midwest,Midwest,OTHF,1.526216e-03
31,Carolinas,"Electric Reliability Council of Texas, Inc.",GAS,8.435249e-10
6,Central,California,OIL,2.409435e-13
55,New York Independent System Operator,Mid-Atlantic,OTHF,4.546099e-06
96,Central,Northwest,OTHF,1.722749e-06
8,California,California,SOLAR,1.031756e-01
113,Mid-Atlantic,Southwest,HYDRO,4.773537e-12
6,New England ISO,California,OIL,2.443830e-21


In [27]:
df.to_csv('regional_trading_fuel_results.csv', index=False)

In [36]:
df.groupby('region')['trading_gen_ratio'].sum()

region
California                                     1.0
Carolinas                                      1.0
Central                                        1.0
Electric Reliability Council of Texas, Inc.    1.0
Florida                                        1.0
Mid-Atlantic                                   1.0
Midwest                                        1.0
New England ISO                                1.0
New York Independent System Operator           1.0
Northwest                                      1.0
Southeast                                      1.0
Southwest                                      1.0
Tennessee Valley Authority                     1.0
Name: trading_gen_ratio, dtype: float64

In [37]:
df.loc[df.region == 'California']

,region,from_region,FuelCategory,trading_gen_ratio
0,California,California,BIOMASS,1.914531e-02
1,California,California,GAS,3.226375e-01
2,California,California,GEOTHERMAL,4.491826e-02
3,California,California,HYDRO,1.324531e-01
4,California,California,NUCLEAR,6.716699e-02
5,California,California,OFSL,2.496135e-04
6,California,California,OIL,1.178448e-04
7,California,California,OTHF,2.017203e-03
8,California,California,SOLAR,1.031756e-01
9,California,California,WIND,5.042608e-02


## Compile flows

In [43]:
flow_df = get_generation_process_df()

Loading 2017 EIA-923 data from csv file
Loading 2017 EIA-860 plant data from csv file
Loading 2015 EIA-923 data from csv file
Loading 2015 EIA-860 plant data from csv file
Loading 2016 EIA-923 data from csv file
Loading 2016 EIA-860 plant data from csv file
Loading 2017 EIA-923 data from csv file
Loading 2017 EIA-860 plant data from csv file
Creating generation process database for Florida ...
Creating generation process database for Midwest ...
Creating generation process database for Mid-Atlantic ...
Creating generation process database for Northwest ...
Creating generation process database for California ...
Creating generation process database for New England ISO ...
Creating generation process database for Carolinas ...
Creating generation process database for Central ...
Creating generation process database for New York Independent System Operator ...
Creating generation process database for Southeast ...
Creating generation process database for Electric Reliability Council of Te

In [44]:
flow_df.sample(5)

,FuelCategory,FlowName,FlowUUID,Compartment,Year,Source,Unit,Subregion,ElementaryFlowPrimeContext,TechnologicalCorrelation,TemporalCorrelation,DataCollection,Emission_factor,Reliability_Score,GeographicalCorrelation,GeomMean,GeomSD,Maximum,Minimum
46795,BIT,Toluene,b43520f0-bcc4-3800-a60e-f8ed389f506a,air,2016,NEI,kg,Tennessee Valley Authority,emission,4.0,1.0,4.0,4.876429e-05,2.891292,1,4.4068722e-05,1.568305044699,7.921216e-05,7.265105e-06
6353,BIOMASS,Beryllium,f0366396-39dc-3c21-9714-ddbb5aef9c4b,air,2016,NEI,kg,Mid-Atlantic,emission,4.0,1.0,4.0,1.729877e-07,1.045520,1,None,None,9.072709e-07,7.593876e-08
8108,BIOMASS,Methylene chloride,efc9d45e-81f9-37bd-84e5-13afda41d6be,air,2016,NEI,kg,California,emission,4.0,1.0,4.0,3.725964e-04,2.101672,1,0.000333769302,1.598617307332,2.116824e-02,1.771885e-06
44844,RC,Formaldehyde,3a4fcbc0-6492-3adf-9514-c3a918eb36a1,air,2016,NEI,kg,Carolinas,emission,4.0,1.0,4.0,2.516078e-05,2.000000,1,2.4146692e-05,1.332213506015,5.534568e-05,5.324369e-06
37191,SUB,Toluene,b43520f0-bcc4-3800-a60e-f8ed389f506a,air,2016,NEI,kg,Central,emission,4.0,1.0,4.0,1.011570e-04,3.650650,1,9.1581319e-05,1.562012428192,9.406947e-04,2.644081e-08


In [45]:
results = pd.merge(df, flow_df, left_on=['FuelCategory', 'from_region'],
                   right_on=['FuelCategory', 'Subregion'])

In [46]:
results.sample(10)

,region,from_region,FuelCategory,trading_gen_ratio,FlowName,FlowUUID,Compartment,Year,Source,Unit,...,TechnologicalCorrelation,TemporalCorrelation,DataCollection,Emission_factor,Reliability_Score,GeographicalCorrelation,GeomMean,GeomSD,Maximum,Minimum
21668,Mid-Atlantic,Central,LIG,5.428701e-07,Copper compounds,6b14b545-b6d7-31d5-bb10-e4ee0a56cda6,air,2016,TRI,kg,...,4.000000,1.0,4.0,1.943348e-05,2.651862,1,1.8730838e-05,1.311784815142,1.094909e-03,4.246685e-06
14048,Northwest,Southwest,RC,1.368336e-04,Di(2-ethylhexyl) phthalate,ff44086f-099e-32d2-9532-3be566000cc4,air,2016,NEI,kg,...,4.000000,1.0,4.0,1.848293e-05,2.468690,1,1.8050387e-05,1.243121699841,2.037273e-05,1.357134e-05
27338,Florida,"Electric Reliability Council of Texas, Inc.",LIG,8.776066e-14,Benzo[ghi]perylene,7e145e90-ed4d-3c84-b19f-a7da241fdb10,air,2016,NEI,kg,...,4.000000,1.0,4.0,7.676645e-09,2.000000,1,7.334e-09,1.352567352204,1.113009e-08,7.669229e-09
18686,New York Independent System Operator,Carolinas,OIL,1.453862e-08,Nitrogen oxides,ff916e13-2e0e-3853-94f8-d7bd94f88f59,air,2016,eGRID,kg,...,4.000000,1.0,4.0,1.436637e+01,2.000000,1,13.469354291866,1.432027096727,3.217986e+01,8.546093e+00
34319,New York Independent System Operator,Mid-Atlantic,OIL,7.880067e-05,Dimethyl sulfate,a26e77ae-ddb4-3e9d-a442-556fcd0a3f3a,air,2016,NEI,kg,...,4.000000,1.0,4.0,9.125748e-06,2.000000,1,None,None,1.033281e-05,1.033281e-05
28173,Florida,"Electric Reliability Council of Texas, Inc.",NUCLEAR,1.257547e-13,Acenaphthylene,316e0c5b-f8a0-3ba5-b33e-0d4724f2e928,air,2016,NEI,kg,...,4.000000,1.0,4.0,3.874086e-10,2.000000,1,None,None,3.874086e-10,3.874086e-10
40925,New England ISO,Midwest,NUCLEAR,1.237010e-06,Nitrogen oxides,ff916e13-2e0e-3853-94f8-d7bd94f88f59,air,2016,eGRID,kg,...,4.000000,1.0,4.0,4.628335e-04,2.000000,1,None,None,4.773727e-03,2.433419e-04
1189,California,California,GAS,3.226375e-01,"Dibenz[a,h]anthracene",d4f8dd30-15a5-3b0b-9661-b2ca97c2f657,air,2016,NEI,kg,...,4.000000,1.0,4.0,1.045544e-08,1.982946,1,8.767e-09,1.810222533079,4.332397e-05,2.319332e-10
37761,Southeast,Midwest,BIT,1.290473e-04,Lead compounds,d13bc6ac-a3b4-3dc8-a265-438200dccef8,air,2016,TRI,kg,...,4.001392,1.0,4.0,1.325283e-05,2.184141,1,1.2965404e-05,1.232951563759,1.681944e-04,1.789246e-06
40043,Mid-Atlantic,Midwest,LIG,1.234882e-04,Di(2-ethylhexyl) phthalate,ff44086f-099e-32d2-9532-3be566000cc4,air,2016,NEI,kg,...,4.002830,1.0,4.0,1.801307e-05,2.159917,1,1.7896729e-05,1.120569836496,5.645759e-05,7.363693e-06


In [47]:
results.to_csv('flows_by_region_trading.gzip', compression='gzip')